In [1]:
import torch
from torch import nn
from torchvision import transforms as T
from PIL import Image
import numpy as np
from pathlib import Path
from collections import deque
import random, datetime, os, copy
%reload_ext autoreload
%autoreload 

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
import gym
from gym.spaces import Box
from gym.wrappers import FrameStack

In [4]:
from nes_py.wrappers import JoypadSpace

In [5]:
import gym_super_mario_bros
import gym_super_mario_bros.actions as JoypadActions

from lib.env_wrappers import EnvWrapperFactory

stateShape = (50, 50)
actionShape = len(JoypadActions.SIMPLE_MOVEMENT)

In [6]:
%reload_ext autoreload
%autoreload 
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, JoypadActions.SIMPLE_MOVEMENT)
env = EnvWrapperFactory.convert(env, shape=stateShape)
state = env.reset()
print(state.shape)

shape before any transformations: (240, 256, 3)
shape after grayscaler: (240, 256)
shape after resizer: (50, 50)
shape after all transformations: (5, 50, 50)
(5, 50, 50)


In [7]:
from agents.ForgetfulAgent import ForgetfulAgent
%reload_ext autoreload
%autoreload 

In [8]:
env.observation_space.shape

(5, 50, 50)

In [9]:
%reload_ext autoreload
%autoreload 
agent = ForgetfulAgent(state_shape=env.observation_space.shape, action_shape=actionShape, device=device)

In [10]:
# try training

maxSteps = 10000
done = True
learnCount = 0

for i in range(maxSteps):
    if done:
        state = env.reset()
    
    action = agent.getAction(state)
    next_state, reward, done, info = env.step(env.action_space.sample())
    
    # add to memory
    agent.cache(state, next_state, action, reward, done)
    
    q, loss = agent.learn()
    
    
    if q is not None:
        learnCount += 1
        if learnCount % 100:
            print(f"q:{q}, loss={loss}")

q:0.07934533059597015, loss=0.36005932092666626
q:1.2445793151855469, loss=0.7474931478500366
q:0.588294267654419, loss=0.384665846824646
q:0.25748515129089355, loss=0.5014095306396484
q:0.06964392215013504, loss=0.24228137731552124
q:0.07162201404571533, loss=0.5473961234092712
q:0.08124406635761261, loss=0.48031964898109436
q:0.13386917114257812, loss=0.3738076090812683
q:0.16510772705078125, loss=0.4633833169937134
q:0.23559226095676422, loss=0.5058587789535522
q:0.2707446813583374, loss=0.638313889503479
q:0.3258936405181885, loss=0.3001350164413452
q:0.40034329891204834, loss=0.432925283908844
q:0.44869476556777954, loss=0.2643730342388153
q:0.4807974696159363, loss=0.2815822660923004
q:0.477449893951416, loss=0.23129650950431824
q:0.48253101110458374, loss=0.37408310174942017
q:0.44777143001556396, loss=0.37128883600234985
q:0.4387868046760559, loss=0.5390638113021851
q:0.3898790776729584, loss=0.49474823474884033
q:0.3756117820739746, loss=0.47101059556007385
q:0.348936855792999